In [1]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import re
import json

# Add the locations of the data folders here, and the crop file locations
seasonal_data_folder = '../data/seasonal_era_agriclimatic/'
ten_day_data_folder = '../data/10-day-data/'
crop_data_2005_file = '../data/spam2005v3r2_global_yield/spam2005V3r2_global_Y_TA.csv'
crop_data_2010_file = '../data/spam2010v1r1_global_yield/spam2010V1r1_global_Y_TA.csv'

# Leave this, it just stores the names of the ten day feature files
ten_day_feature_files = {
    'BEDD': 'BEDD_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'FD': 'FD_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'R20mm': 'R20mm_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'R10mm': 'R10mm_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'ID': 'ID_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TG': 'TG_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TN': 'TN_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'DTR': 'DTR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'RR1': 'RR1_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'RR': 'RR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'SDII': 'SDII_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'SU': 'SU_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TG': 'TG_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TNn': 'TNn_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TR': 'TR_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TX': 'TX_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TXn': 'TXn_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc',
    'TXx': 'TXx_C3S-glob-agric_WFDEI_hist_dek_19810101-20101231_v1.nc'
}

# I noticed the CSU file is actually the same as the CSDI file, so I havent included it
seasonal_feature_files = {
    'CDD': 'CDD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CFD': 'CFD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CWD': 'CWD_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'WW': 'WW_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'WSDI': 'WSDI_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc',
    'CSDI': 'CSDI_C3S-glob-agric_WFDEI_hist_season_19810101-20101231_v1.nc'
}

/home/edward_brown/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# Create crop data file and clean it
crops = ['maize']
crop_data_2005 = pd.read_csv(crop_data_2005_file,  encoding = "ISO-8859-1")
crop_data_2010 = pd.read_csv(crop_data_2010_file,  encoding = "ISO-8859-1")

merged = crop_data_2010.merge(crop_data_2005, how='inner', on='alloc_key', suffixes=['_2010', '_2005'])
kept_columns = ['alloc_key', 'x', 'y', 'iso3_2010']
for crop in crops:
    kept_columns += [f'{crop[0:4]}_a_2010', f'{crop[0:4]}_a_2005']
merged = merged[kept_columns]
for crop in crops:
    merged = merged.rename(columns={
    f'{crop[0:4]}_a_2010': f'{crop}_a_2010',
    f'{crop[0:4]}_a_2005': f'{crop}_a_2005',
})
    
merged = merged.rename(columns={
    'iso3_2010': 'iso3',
    'x': 'lon',
    'y': 'lat'
})

del crop_data_2005
del crop_data_2010
data = merged.dropna()
del merged

# Only include non zero yields
data = data[data['maize_a_2005'] > 0]
data = data[data['maize_a_2010'] > 0]
data

,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005
0,4383640,123.291667,53.541667,CHN,3918.1,2819.2
2,4393629,122.375000,53.458333,CHN,3119.3,2028.5
7,4403648,123.958333,53.375000,CHN,3230.1,2337.9
8,4403649,124.041667,53.375000,CHN,3119.3,2010.2
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2
...,...,...,...,...,...,...
665862,7670982,-98.208333,26.125000,USA,5939.7,6106.9
665863,7670983,-98.125000,26.125000,USA,6050.9,6107.8
665864,7670984,-98.041667,26.125000,USA,6111.0,6131.7
665865,7670985,-97.958333,26.125000,USA,6152.1,6181.2


In [8]:
def compute_seasonal_feature(feature, file_location, year, dataset, interpolation_method='linear'):
    lats = xr.DataArray(list(dataset['lat'].values), dims='z')
    lons = xr.DataArray(list(dataset['lon'].values), dims='z')
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    quarter_time_mapping = {
        'Q1': '01-16',
        'Q2': '04-16',
        'Q3': '07-16',
        'Q4': '10-16'
    }
    with xr.open_dataset(file_location) as ds:
        feature_data = ds.load()
    for quarter in quarters:
        feature_name = f'{feature}-{quarter}-{year}'
        time = f'{year}-{quarter_time_mapping[quarter]}'
        timed = feature_data.sel(time=time).squeeze()
        feature_interpolated = timed.interp(lat=lats, lon=lons, method=interpolation_method)
        dataset[feature_name] = getattr(feature_interpolated, feature)
    return dataset

In [4]:
# Compute the seasonal features        
years = ['2010', '2005']
seasonal_features = [
    'CDD',
    'CFD',
    'CWD',
    'WW',
    'WSDI',
    'CSDI'
]

for feature in tqdm(seasonal_features):
    file_location = seasonal_data_folder + seasonal_feature_files[feature]
    for year in years:
        data = compute_seasonal_feature(feature, file_location, year, data)
data.head(10)

100%|██████████| 6/6 [00:07<00:00,  1.32s/it]


,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,...,WSDI-Q3-2005,WSDI-Q4-2005,CSDI-Q1-2010,CSDI-Q2-2010,CSDI-Q3-2010,CSDI-Q4-2010,CSDI-Q1-2005,CSDI-Q2-2005,CSDI-Q3-2005,CSDI-Q4-2005
0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,27.0000,29.194444,9.048611,16.583333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
2,4393629,122.375000,53.458333,CHN,3119.3,2028.5,27.0000,24.604167,8.687500,17.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,3.375,0.0,0.0,0.0
7,4403648,123.958333,53.375000,CHN,3230.1,2337.9,24.8125,26.000000,9.250000,16.562500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
8,4403649,124.041667,53.375000,CHN,3119.3,2010.2,23.9375,26.000000,9.250000,16.687500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,27.0000,26.326389,9.000000,16.465278,...,0.0,0.0,0.0,0.0,0.0,0.0,2.500,0.0,0.0,0.0
10,4413638,123.125000,53.291667,CHN,3918.1,2819.2,27.0000,27.395833,9.000000,16.312500,...,0.0,0.0,0.0,0.0,0.0,0.0,1.500,0.0,0.0,0.0
53,4553678,126.458333,52.125000,CHN,3288.5,1848.2,20.0000,41.500000,9.854167,25.062500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
55,4563678,126.458333,52.041667,CHN,4044.0,1897.4,20.0000,43.833333,9.756944,25.937500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
56,4603680,126.625000,51.708333,CHN,1611.0,598.8,20.0000,43.916667,9.770833,28.916667,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0
58,4613680,126.625000,51.625000,CHN,3943.7,2416.1,20.0000,43.750000,9.812500,28.750000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0


In [9]:
# Compute the 10 day features
def compute_ten_day_feature(feature, file_location, year, dataset, interpolation_method='linear'):
    days = ['05', '15', '25']
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    lats_ = xr.DataArray(list(data['lat'].values), dims='z')
    lons_ = xr.DataArray(list(data['lon'].values), dims='z')
    with xr.open_dataset(file_location) as ds:
        feature_data = ds.load()
        for day in days:
            for month in months:
                time = f'{year}-{month}-{day}'
                timed = feature_data.sel(time=time).squeeze()
                feature_interpolated = timed.interp(lat=lats_, lon=lons_, method='linear')
                dataset[f'{feature}-{month}-{day}-{year}'] = getattr(feature_interpolated, feature)

    return dataset

In [7]:
# Choose the features you want at the top of the notebook based on the ten_day_feature_files
# dict, and put them in this array.
ten_day_features = [
    'BEDD',
    'FD',
    'R20mm',
    'RR1',
    'DTR'
]
years = ['2010', '2005']

for feature in tqdm(ten_day_features):
    file_location = ten_day_data_folder + ten_day_feature_files[feature]
    for year in years:
        data = compute_ten_day_feature(feature, file_location, year, data, interpolation_method='linear')
data.head(10)

100%|██████████| 5/5 [00:50<00:00, 10.01s/it]


,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,CDD-Q4-2010,...,DTR-03-25-2005,DTR-04-25-2005,DTR-05-25-2005,DTR-06-25-2005,DTR-07-25-2005,DTR-08-25-2005,DTR-09-25-2005,DTR-10-25-2005,DTR-11-25-2005,DTR-12-25-2005
0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,27.0000,29.194444,9.048611,16.583333,...,18.507736,15.031678,17.593290,15.704611,18.523022,14.092479,15.016566,13.988310,18.100036,12.329345
2,4393629,122.375000,53.458333,CHN,3119.3,2028.5,27.0000,24.604167,8.687500,17.000000,...,18.651182,15.191576,16.881676,16.390320,18.770228,14.457341,15.568386,14.757354,18.749596,12.080565
7,4403648,123.958333,53.375000,CHN,3230.1,2337.9,24.8125,26.000000,9.250000,16.562500,...,18.019471,14.658110,18.091173,15.529109,18.057473,14.120732,14.818996,13.382273,17.323588,12.099928
8,4403649,124.041667,53.375000,CHN,3119.3,2010.2,23.9375,26.000000,9.250000,16.687500,...,17.901220,14.626593,18.110101,15.483601,17.987763,14.092668,14.805402,13.276832,17.190924,12.048447
9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,27.0000,26.326389,9.000000,16.465278,...,19.109965,15.011482,17.723100,15.926385,18.664236,14.253544,15.248663,14.395436,18.549792,11.976130
10,4413638,123.125000,53.291667,CHN,3918.1,2819.2,27.0000,27.395833,9.000000,16.312500,...,19.094177,15.011628,17.769180,15.865833,18.617434,14.226244,15.185990,14.324799,18.457578,11.970339
53,4553678,126.458333,52.125000,CHN,3288.5,1848.2,20.0000,41.500000,9.854167,25.062500,...,13.891450,11.883485,16.274661,15.594074,15.482645,12.293894,14.238087,11.699558,14.223947,12.616824
55,4563678,126.458333,52.041667,CHN,4044.0,1897.4,20.0000,43.833333,9.756944,25.937500,...,13.965520,11.814335,16.225528,15.589258,15.409521,12.255848,14.201804,11.760957,14.234442,12.807903
56,4603680,126.625000,51.708333,CHN,1611.0,598.8,20.0000,43.916667,9.770833,28.916667,...,14.011786,11.552213,15.871668,15.422135,14.860714,11.928108,14.028940,11.860447,14.068478,13.385260
58,4613680,126.625000,51.625000,CHN,3943.7,2416.1,20.0000,43.750000,9.812500,28.750000,...,14.019536,11.606706,15.795987,15.397675,14.716153,11.918716,13.992922,11.876829,14.113375,13.509021


In [8]:
# Save data to a csv
file_path = f'head_of_soils_recommendations.csv'
data.to_csv(file_path)

In [9]:
# Inspect a CSV file
file_path = f'head_of_soils_recommendations.csv'
data = pd.read_csv(file_path)
data.head()

,Unnamed: 0,alloc_key,lon,lat,iso3,maize_a_2010,maize_a_2005,CDD-Q1-2010,CDD-Q2-2010,CDD-Q3-2010,...,DTR-03-25-2005,DTR-04-25-2005,DTR-05-25-2005,DTR-06-25-2005,DTR-07-25-2005,DTR-08-25-2005,DTR-09-25-2005,DTR-10-25-2005,DTR-11-25-2005,DTR-12-25-2005
0,0,4383640,123.291667,53.541667,CHN,3918.1,2819.2,27.0000,29.194444,9.048611,...,18.507736,15.031678,17.593290,15.704611,18.523022,14.092479,15.016566,13.988310,18.100036,12.329345
1,2,4393629,122.375000,53.458333,CHN,3119.3,2028.5,27.0000,24.604167,8.687500,...,18.651182,15.191576,16.881676,16.390320,18.770228,14.457341,15.568386,14.757354,18.749596,12.080565
2,7,4403648,123.958333,53.375000,CHN,3230.1,2337.9,24.8125,26.000000,9.250000,...,18.019471,14.658110,18.091173,15.529109,18.057473,14.120732,14.818996,13.382273,17.323588,12.099928
3,8,4403649,124.041667,53.375000,CHN,3119.3,2010.2,23.9375,26.000000,9.250000,...,17.901220,14.626593,18.110101,15.483601,17.987763,14.092668,14.805402,13.276832,17.190924,12.048447
4,9,4413637,123.041667,53.291667,CHN,3918.1,2819.2,27.0000,26.326389,9.000000,...,19.109965,15.011482,17.723100,15.926385,18.664236,14.253544,15.248663,14.395436,18.549792,11.976130


In [2]:
# Create a custom train and test set adding in the features you want. It reads an already computed 
# dataset but you can then remove features to see the effect on score.
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random
import re
import json
import pickle
import os

# Choose the run ID for this training set
run_id = 42


run_folder = 'run_data'
regressors_folder = 'regressors'

if not os.path.exists(run_folder):
    os.makedirs(run_folder)
if not os.path.exists(regressors_folder):
    os.makedirs(regressors_folder)

# Which data set are we reading in?
read_file_path = f'head_of_soils_recommendations.csv'

# Set the random seed for the whole process of creating the training and test set.
random_seed = run_id
random.seed(a=random_seed)
np.random.seed(random_seed)

# Test set fraction. What fraction of the data set to make a test set
test_size = 0.2

# Only read a sample fraction of the data, speeds things up, but you'll get worse performance.
# Use 1.0 as the sample fraction to get the full data set
sample_fraction = 1.0
data = pd.read_csv(read_file_path, skiprows=lambda i: i>0 and random.random() > sample_fraction)

# Remove datapoints where yield changes by more than the fraction between 2005 and 2010.
# This is unlikely to be due to changes in climate.
limit_change_in_yield = False
limit = 0.5

# Which type of features to include (if True must already exist in read dataset)
include_coordinates = True
include_2005_maize = True
include_seasonal = True
include_ten_day = True
include_2005_climate = True
include_soil_type = False
include_growing_zones = False

# Drop nans?
drop_nans = True

# Choose a subset of your features! (must already exist in original file)
ten_day_features = [
    'BEDD',
    'R20mm',
    'DTR',
    'RR1',
    'FD'
]

seasonal_features = [
    'CDD',
    'CFD',
    'CWD',
    'WW',
    'WSDI',
    'CSDI'
]

############################################################
############################################################


if limit_change_in_yield:
    data = data[abs(data['maize_a_2010'] - data['maize_a_2005']) / data['maize_a_2005'] < limit]

features = []

if include_growing_zones:
    growing_zones = [
        'Inland water bodies',
        'Subtropics - summer rainfall',
        'Subtropics - winter rainfall',
        'Temperature - continental',
        'Temperature - oceanic',
        'Temperature - subcontinental',
        'Tropics'
    ]
    
if include_soil_type:
    features = features + ['soil_type']

if include_2005_climate:
    years = ['2010', '2005']
else:
    years = ['2010']

if include_seasonal:
    quarters = ['Q1', 'Q2', 'Q3', 'Q4']
    for feature in seasonal_features:
        for quarter in quarters:
            for year in years:
                features.append(f'{feature}-{quarter}-{year}')

if include_ten_day:
    days = ['05', '15', '25']
    months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
    for feature in ten_day_features:
        for day in days:
            for month in months:
                for year in years:
                    features.append(f'{feature}-{month}-{day}-{year}')

if include_coordinates:
    features = features + ['lon', 'lat']
    
if include_2005_maize:
    features = features + ['maize_a_2005']

target = 'maize_a_2010'

if drop_nans:
    data = data.dropna()
    
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_seed)

/home/edward_brown/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [6]:
# Try a simple random forest regressor
regressor = RandomForestRegressor(n_estimators=5, n_jobs=10, random_state=random_seed, verbose=3)
regressor.fit(X_train, y_train)
score = regressor.score(X_test, y_test)
regressor_filename = f'{regressors_folder}/regressor_run_id_{run_id}.pkl'
pickle.dump(regressor, open(regressor_filename, 'wb'))

run_data = {
    'ten_day_features': ten_day_features,
    'seasonal_features': seasonal_features,
    'limit_change_in_yield': limit_change_in_yield,
    'limit': limit,
    'test_size': test_size,
    'sample_fraction': sample_fraction,
    'random_seed': random_seed,
    'include_coordinates': include_coordinates,
    'include_2005_maize': include_2005_maize,
    'include_seasonal': include_seasonal,
    'include_ten_day': include_ten_day,
    'include_2005_climate': include_2005_climate,
    'include_soil_type': include_soil_type,
    'include_growing_zones': include_growing_zones,
    'run_id': run_id,
    'target': target,
    'features': features,
    'read_file_path': read_file_path,
    'drop_nans': drop_nans,
    'regressor_path': regressor_filename,
    'regressor_score': score
}
with open(f'{run_folder}/run_id_{run_id}.json', 'w') as fp:
    json.dump(run_data, fp)
del X
del y
del data

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from tqdm import tqdm
tqdm.pandas()
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import random
import re
import json

crop = 'maize'
# Compute indicators on Hadgem data
# Change the prediction id to not overun previous saved data
run_id = 42

with open(f'{run_folder}/run_id_{run_id}.json', 'r') as fp:
    run_data = json.load(fp)

regressor_file_name = f'{regressors_folder}/regressor_run_id_{run_id}.pkl'
regressor = pickle.load(open(regressor_file_name, 'rb'))

file_path = run_data['read_file_path']
data = pd.read_csv(file_path)

prediction_year = 2039
rcp = '8p5'
folder = f'../data/hadgem_rcp_{rcp}_agroclimatic_indicators/'
out_file = f'../data/hadgem_predictions_rcp_{rcp}_year_{prediction_year}_run_id_{run_id}.csv'
current_year = 2020
seasonal_features = run_data['seasonal_features']
ten_day_features = run_data['ten_day_features']

for year in [prediction_year, current_year]:
    if int(year) in range(2011,2041):
        file_ending = '20110101-20401231_v1.nc'
    elif int(year) in range(2041,2071):
        file_ending = '20410101-20701231_v1.nc'
    for feature in tqdm(seasonal_features):
        file_location = f'{folder}{feature}_C3S-glob-agric_hadgem2-es_rcp{rcp}_season_{file_ending}'
        data = compute_seasonal_feature(feature, file_location, str(year), data)

    for feature in tqdm(ten_day_features):    
        file_location = f'{folder}{feature}_C3S-glob-agric_hadgem2-es_rcp{rcp}_dek_{file_ending}'
        data = compute_ten_day_feature(feature, file_location, str(year), data)

    data = data.dropna()
    year_features = [s.replace('2010', f'{year}') for s in run_data['features']]
    data[f'{year}_predict'] = regressor.predict(data[year_features])
    
data['Yield Percent Change From Current'] = 100 * (data[f'{prediction_year}_predict'] - data[f'{current_year}_predict'])/data[f'{current_year}_predict']
data['Yield Percent Change From 2010'] = 100 * (data['maize_a_2010'] - data[f'{current_year}_predict'])/data[f'{current_year}_predict']

out_data = data[['lat', 'lon', f'{crop}_a_2010', f'{current_year}_predict', f'{prediction_year}_predict', 'Yield Percent Change']].to_csv(out_file)

/home/edward_brown/anaconda3/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|██████████| 5/5 [00:26<00:00,  5.32s/it]
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
100%|██████████| 5/5 [00:28<00:00,  5.71s/it]
[Parallel(n_jobs=5)]: Using backend ThreadingBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.1s finished
